In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [5]:
pdf=pd.read_csv("insurance_data (1).csv")

In [7]:
pdf

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(pdf[["age","affordibility"]],pdf["bought_insurance"],test_size=.2)

In [31]:
x_train_copy=x_train.copy()
x_train_copy['age']=x_train_copy["age"]/100
x_test_copy=x_test.copy()
x_test_copy['age']=x_test_copy["age"]/100



In [33]:
print(x_train_copy.shape)

(22, 2)


In [42]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation="sigmoid", 
                       kernel_initializer='ones', bias_initializer='zeros')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model
model.fit(x_train_copy, y_train, epochs=5000)

Epoch 1/5000


C:\Users\pc\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step - accuracy: 0.4545 - loss: 0.7602
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4545 - loss: 0.7597
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4545 - loss: 0.7592
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.4545 - loss: 0.7587
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4545 - loss: 0.7583
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4545 - loss: 0.7578
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4545 - loss: 0.7573
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4545 - loss: 0.7568
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.4545 - loss: 0.7563
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.4545 - loss: 0.7558
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.4545 - loss: 0.7554
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4545 - 

In [44]:
model.evaluate(x_test_copy,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step - accuracy: 0.8333 - loss: 0.4359


[0.4359000027179718, 0.8333333134651184]

In [46]:
coef,intercept=model.get_weights()

In [40]:
coef,intercept

(array([[0.84534484],
        [0.6965144 ]], dtype=float32),
 array([-0.71857727], dtype=float32))

In [103]:
def sigmoid_numpy(x):
    import math
    return 1/(1+np.exp(-x))
sigmoid(18)

0.9999999847700205

In [50]:
x_test

,age,affordibility
21,26,0
22,40,1
25,54,1
19,18,1
7,60,0
5,56,1


##### instead of model.predict let us write our own prediction function that uses w1,w2 and bias

In [53]:
def prediction_function(age,affordibility):
    weighted_sum=coef[0]*age+coef[1]*affordibility+intercept
    return sigmoid(weighted_sum)
prediction_function(.47,1)

C:\Users\pc\AppData\Local\Temp\ipykernel_23120\574919516.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.65732794092352

 Now we start implementing gradient descent in plain python. Again the goal is to come up
 with same WI, w2 and bias that keras model calculated. We want to show how
 keras/tensorflow would have computed these values internally using gradient descent
First write couple of helper routinB such as sigmoid and log_loss

In [63]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

All right now comes the time to implement our final gradient descent function !! hurray!!

In [105]:

def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [107]:
gradient_descent(x_train_copy["age"],x_train_copy["affordibility"],y_train,1000, 0.4631)

Epoch:0, w1:0.9652761325686007, w2:0.9303255150604461, bias:-0.13949455346536418, loss:0.7601984732642247
Epoch:1, w1:0.9377574845249491, w2:0.8733983120145848, bias:-0.26036255005612674, loss:0.7128335374980371
Epoch:2, w1:0.9169317614968799, w2:0.828516996930195, bias:-0.3640549785425895, loss:0.6785285724028524
Epoch:3, w1:0.9020716246441389, w2:0.7944839192481573, bias:-0.45251836347734337, loss:0.6542991820909174
Epoch:4, w1:0.8923572832638199, w2:0.7698539843940485, bias:-0.5278885349072521, loss:0.6374348458483953
Epoch:5, w1:0.8869733842241523, w2:0.7531401798272133, bias:-0.5922563293777852, loss:0.6257335108119855
Epoch:6, w1:0.885169295933489, w2:0.7429468692711269, bias:-0.6475267383722412, loss:0.6175423838143119
Epoch:7, w1:0.8862876379345218, w2:0.7380374923792423, bias:-0.6953561845315798, loss:0.6116880691650164
Epoch:8, w1:0.8897712832196315, w2:0.7373565202684701, bias:-0.737141354274553, loss:0.6073689742044183
Epoch:9, w1:0.8951582115862454, w2:0.7400248076508356, 

(4.245179872318476, 1.7465936662020536, -3.098808596487946)